In [1]:
#Reading appropriate libraries 

import tensorflow as tf
from tensorflow import keras

#Library for binarized neural network
import larq as lq
import pandas as pd
import numpy as np
import random as rand
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

2022-12-02 14:44:03.929657: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 14:44:04.642650: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/saumya/.conda/envs/tf/lib/
2022-12-02 14:44:04.642745: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/saumya/.conda/envs/tf/lib/
2022-12-02 14:44:04.642754: W tensorflow/compiler/tf2tensorrt/

In [2]:
#Reading banknote authentication dataset for this code

df = pd.read_csv('/home/saumya/Downloads/banknote_authentication.csv', sep = ';')

In [3]:
#Extracting the needed information
xdata = df[['wavelet_transformed_variance', 'wavelet_transformed_skewness', 'wavelet_transformed_curtosis','image_entropy']].to_numpy()
ydata = df[['counterfeit']].to_numpy()

In [4]:
#Here the error from the approximate processors are simulated in the dataset.
#Here we are simulating a random error of +,- 10% 
def add_error(xdata):
    error_mat = np.ones(np.shape(xdata))
    for i in range(error_mat.shape[0]):
        for j in range(error_mat.shape[1]):
            error_mat[i][j] = xdata[i][j]*0.1*rand.random()*(-1)**(rand.randint(0, 1))
    return(error_mat)

In [5]:
#new data with added error on it
#simulating error for 10 approximate process
for i in range(10):
    error_mat = add_error(xdata)
    xdata_error = xdata + error_mat

In [6]:
#Fitting the data 
scaler = StandardScaler()
xtrain = scaler.fit_transform(xdata)
xtrain_error = scaler.fit_transform(xdata_error)

In [7]:
#Splitting the data for trainning and testing
xtrain, xtest, ytrain, ytest = train_test_split(xdata, ydata, test_size = 0.2, train_size=0.8)
xtrain_error, xtest_error, ytrain_error, ytest_error = train_test_split(xdata_error, ydata, test_size = 0.2, train_size=0.8)

In [8]:
# All quantized layers except the first will use the same options
kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip")

#Defining model with no added error
model = tf.keras.models.Sequential()

# In the first layer we only quantize the weights and not the input
#Input Layer
model.add(lq.layers.QuantDense(4, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
#Second Layer
model.add(lq.layers.QuantDense(8, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
#Third Layer
model.add(lq.layers.QuantDense(4, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
#Output layer
model.add(lq.layers.QuantDense(1,activation='sigmoid', use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))

2022-12-02 14:44:06.094345: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 14:44:06.124369: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 14:44:06.124570: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 14:44:06.125329: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [9]:
#Defining the model to train with error added data

model_error = tf.keras.models.Sequential()

# In the first layer we only quantize the weights and not the input
#Input Layer
model_error.add(lq.layers.QuantDense(4, use_bias=False, **kwargs))
model_error.add(tf.keras.layers.BatchNormalization(scale=False))
#Second Layer
model_error.add(lq.layers.QuantDense(8, use_bias=False, **kwargs))
model_error.add(tf.keras.layers.BatchNormalization(scale=False))
#Third Layer
model_error.add(lq.layers.QuantDense(4, use_bias=False, **kwargs))
model_error.add(tf.keras.layers.BatchNormalization(scale=False))
#Output layer
model_error.add(lq.layers.QuantDense(1,activation='sigmoid', use_bias=False, **kwargs))
model_error.add(tf.keras.layers.BatchNormalization(scale=False))

In [10]:
eph_step = 2000

In [11]:
#Compiling the model with no added error

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(xtrain, ytrain, batch_size=64, epochs=eph_step)

test_loss, test_acc = model.evaluate(xtest, ytest)

Epoch 1/2000
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2022-12-02 14:44:08.747279: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fd47c0179a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-12-02 14:44:08.747312: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2022-12-02 14:44:08.752437: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2022-12-02 14:44:08.817177: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-02 14:44:08.853697: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


18/18 [==============================] - 3s 4ms/step - loss: 4.2455 - accuracy: 0.5834
Epoch 2/2000
18/18 [==============================] - 0s 4ms/step - loss: 4.1515 - accuracy: 0.6217
Epoch 3/2000
18/18 [==============================] - 0s 3ms/step - loss: 4.9110 - accuracy: 0.6071
Epoch 4/2000
18/18 [==============================] - 0s 4ms/step - loss: 4.1289 - accuracy: 0.6080
Epoch 5/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.6430 - accuracy: 0.6098
Epoch 6/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.4701 - accuracy: 0.6126
Epoch 7/2000
18/18 [==============================] - 0s 4ms/step - loss: 4.0729 - accuracy: 0.7284
Epoch 8/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.5659 - accuracy: 0.7311
Epoch 9/2000
18/18 [==============================] - 0s 3ms/step - loss: 3.6204 - accuracy: 0.7411
Epoch 10/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.5316 - accuracy: 0.7356
Epoch 11/200

18/18 [==============================] - 0s 4ms/step - loss: 3.7860 - accuracy: 0.7429
Epoch 83/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.9491 - accuracy: 0.7384
Epoch 84/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.7577 - accuracy: 0.7457
Epoch 85/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.9308 - accuracy: 0.7338
Epoch 86/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.7900 - accuracy: 0.7311
Epoch 87/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.8327 - accuracy: 0.7356
Epoch 88/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.6866 - accuracy: 0.7247
Epoch 89/2000
18/18 [==============================] - 0s 4ms/step - loss: 4.0496 - accuracy: 0.7338
Epoch 90/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.5167 - accuracy: 0.7138
Epoch 91/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.6474 - accuracy: 0.7539
Epoc

18/18 [==============================] - 0s 4ms/step - loss: 3.8506 - accuracy: 0.7375
Epoch 163/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.6654 - accuracy: 0.7411
Epoch 164/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.3800 - accuracy: 0.7475
Epoch 165/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.5615 - accuracy: 0.7484
Epoch 166/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.8161 - accuracy: 0.7457
Epoch 167/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.7552 - accuracy: 0.7411
Epoch 168/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.4573 - accuracy: 0.7566
Epoch 169/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.7693 - accuracy: 0.7366
Epoch 170/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.5723 - accuracy: 0.7466
Epoch 171/2000
18/18 [==============================] - 0s 4ms/step - loss: 3.4374 - accuracy: 0.

...

18/18 [==============================] - 0s 4ms/step - loss: 1.0459 - accuracy: 0.7092
Epoch 1675/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.2119 - accuracy: 0.7265
Epoch 1676/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.0842 - accuracy: 0.7256
Epoch 1677/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.1966 - accuracy: 0.7256
Epoch 1678/2000
18/18 [==============================] - 0s 4ms/step - loss: 0.8453 - accuracy: 0.7165
Epoch 1679/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.0419 - accuracy: 0.7110
Epoch 1680/2000
18/18 [==============================] - 0s 3ms/step - loss: 1.5468 - accuracy: 0.6992
Epoch 1681/2000
18/18 [==============================] - 0s 4ms/step - loss: 0.8468 - accuracy: 0.6919
Epoch 1682/2000
18/18 [==============================] - 0s 4ms/step - loss: 0.8037 - accuracy: 0.7366
Epoch 1683/2000
18/18 [==============================] - 0s 4ms/step - loss: 0.8904 - acc

18/18 [==============================] - 0s 4ms/step - loss: 1.6661 - accuracy: 0.6609
Epoch 1754/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4101 - accuracy: 0.6217
Epoch 1755/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.6198 - accuracy: 0.7347
Epoch 1756/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.2857 - accuracy: 0.7156
Epoch 1757/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3836 - accuracy: 0.7156
Epoch 1758/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4815 - accuracy: 0.7502
Epoch 1759/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4952 - accuracy: 0.7648
Epoch 1760/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5668 - accuracy: 0.7666
Epoch 1761/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4782 - accuracy: 0.7502
Epoch 1762/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4821 - acc

18/18 [==============================] - 0s 4ms/step - loss: 1.5328 - accuracy: 0.7803
Epoch 1833/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5451 - accuracy: 0.7639
Epoch 1834/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5114 - accuracy: 0.7748
Epoch 1835/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5402 - accuracy: 0.7858
Epoch 1836/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5941 - accuracy: 0.7557
Epoch 1837/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5536 - accuracy: 0.7785
Epoch 1838/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.5150 - accuracy: 0.7748
Epoch 1839/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.5162 - accuracy: 0.7840
Epoch 1840/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.5671 - accuracy: 0.7675
Epoch 1841/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.5311 - acc

18/18 [==============================] - 0s 4ms/step - loss: 1.5585 - accuracy: 0.7703
Epoch 1912/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.5616 - accuracy: 0.7758
Epoch 1913/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4831 - accuracy: 0.7521
Epoch 1914/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5694 - accuracy: 0.7666
Epoch 1915/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5295 - accuracy: 0.7712
Epoch 1916/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.5580 - accuracy: 0.7557
Epoch 1917/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.5395 - accuracy: 0.7794
Epoch 1918/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.5215 - accuracy: 0.7685
Epoch 1919/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.5583 - accuracy: 0.7694
Epoch 1920/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.5484 - acc

18/18 [==============================] - 0s 5ms/step - loss: 1.4787 - accuracy: 0.7521
Epoch 1991/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.4816 - accuracy: 0.7657
Epoch 1992/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.5084 - accuracy: 0.7274
Epoch 1993/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.4560 - accuracy: 0.7420
Epoch 1994/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.2539 - accuracy: 0.7530
Epoch 1995/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.4552 - accuracy: 0.7712
Epoch 1996/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.3251 - accuracy: 0.7521
Epoch 1997/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.3140 - accuracy: 0.7356
Epoch 1998/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.4915 - accuracy: 0.8004
Epoch 1999/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.6230 - acc

In [12]:
#Compiling the model with added error

model_error.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_error.fit(xtrain_error, ytrain_error, batch_size=64, epochs=eph_step)

test_loss_error, test_acc_error = model_error.evaluate(xtest_error, ytest_error)

Epoch 1/2000
18/18 [==============================] - 2s 5ms/step - loss: 6.5858 - accuracy: 0.4330
Epoch 2/2000
18/18 [==============================] - 0s 4ms/step - loss: 6.1723 - accuracy: 0.4676
Epoch 3/2000
18/18 [==============================] - 0s 4ms/step - loss: 7.1076 - accuracy: 0.4303
Epoch 4/2000
18/18 [==============================] - 0s 5ms/step - loss: 7.4655 - accuracy: 0.4740
Epoch 5/2000
18/18 [==============================] - 0s 5ms/step - loss: 6.3364 - accuracy: 0.4585
Epoch 6/2000
18/18 [==============================] - 0s 5ms/step - loss: 6.8168 - accuracy: 0.4594
Epoch 7/2000
18/18 [==============================] - 0s 4ms/step - loss: 7.0685 - accuracy: 0.4549
Epoch 8/2000
18/18 [==============================] - 0s 4ms/step - loss: 7.0360 - accuracy: 0.4740
Epoch 9/2000
18/18 [==============================] - 0s 4ms/step - loss: 7.4006 - accuracy: 0.4558
Epoch 10/2000
18/18 [==============================] - 0s 4ms/step - loss: 6.9813 - accuracy: 0.4403

18/18 [==============================] - 0s 4ms/step - loss: 1.9523 - accuracy: 0.7821
Epoch 83/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.6491 - accuracy: 0.6463
Epoch 84/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.7672 - accuracy: 0.6992
Epoch 85/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.7612 - accuracy: 0.7156
Epoch 86/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.1501 - accuracy: 0.7721
Epoch 87/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.1655 - accuracy: 0.7730
Epoch 88/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.3376 - accuracy: 0.7621
Epoch 89/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.0633 - accuracy: 0.7648
Epoch 90/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.1786 - accuracy: 0.7794
Epoch 91/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.3982 - accuracy: 0.7803
Epoc

18/18 [==============================] - 0s 4ms/step - loss: 2.0580 - accuracy: 0.7530
Epoch 163/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.7825 - accuracy: 0.7630
Epoch 164/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.8386 - accuracy: 0.7794
Epoch 165/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.2397 - accuracy: 0.7329
Epoch 166/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.9995 - accuracy: 0.7302
Epoch 167/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.8864 - accuracy: 0.7274
Epoch 168/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.5913 - accuracy: 0.7119
Epoch 169/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.6303 - accuracy: 0.7138
Epoch 170/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.5918 - accuracy: 0.7129
Epoch 171/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.0112 - accuracy: 0.

18/18 [==============================] - 0s 4ms/step - loss: 2.0572 - accuracy: 0.7530
Epoch 243/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.7894 - accuracy: 0.7539
Epoch 244/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.2584 - accuracy: 0.7192
Epoch 245/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.6698 - accuracy: 0.7329
Epoch 246/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.2376 - accuracy: 0.7876
Epoch 247/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.2040 - accuracy: 0.7685
Epoch 248/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4323 - accuracy: 0.7830
Epoch 249/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3423 - accuracy: 0.7858
Epoch 250/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.2615 - accuracy: 0.7785
Epoch 251/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.1503 - accuracy: 0.

18/18 [==============================] - 0s 4ms/step - loss: 1.5367 - accuracy: 0.7867
Epoch 323/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.6563 - accuracy: 0.7995
Epoch 324/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.7904 - accuracy: 0.7840
Epoch 325/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.6510 - accuracy: 0.7912
Epoch 326/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.4799 - accuracy: 0.7912
Epoch 327/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.4519 - accuracy: 0.7858
Epoch 328/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.6488 - accuracy: 0.7758
Epoch 329/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.7485 - accuracy: 0.7821
Epoch 330/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.7987 - accuracy: 0.7694
Epoch 331/2000
18/18 [==============================] - 0s 6ms/step - loss: 1.4946 - accuracy: 0.

...

18/18 [==============================] - 0s 4ms/step - loss: 1.7478 - accuracy: 0.7046
Epoch 1675/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.6984 - accuracy: 0.6964
Epoch 1676/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.8919 - accuracy: 0.7019
Epoch 1677/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.7759 - accuracy: 0.6892
Epoch 1678/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.6996 - accuracy: 0.6554
Epoch 1679/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.7885 - accuracy: 0.7201
Epoch 1680/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4945 - accuracy: 0.6764
Epoch 1681/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.9997 - accuracy: 0.6901
Epoch 1682/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.7692 - accuracy: 0.6563
Epoch 1683/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.6009 - acc

18/18 [==============================] - 0s 4ms/step - loss: 1.6091 - accuracy: 0.7867
Epoch 1754/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5283 - accuracy: 0.7621
Epoch 1755/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.6706 - accuracy: 0.7812
Epoch 1756/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5564 - accuracy: 0.7858
Epoch 1757/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5255 - accuracy: 0.7721
Epoch 1758/2000
18/18 [==============================] - 0s 3ms/step - loss: 1.6784 - accuracy: 0.7767
Epoch 1759/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.5468 - accuracy: 0.7311
Epoch 1760/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.6100 - accuracy: 0.7694
Epoch 1761/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.6489 - accuracy: 0.7858
Epoch 1762/2000
18/18 [==============================] - 0s 3ms/step - loss: 1.6183 - acc

18/18 [==============================] - 0s 4ms/step - loss: 1.4232 - accuracy: 0.7867
Epoch 1833/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3262 - accuracy: 0.7840
Epoch 1834/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3294 - accuracy: 0.7849
Epoch 1835/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4911 - accuracy: 0.7931
Epoch 1836/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3180 - accuracy: 0.7821
Epoch 1837/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.4936 - accuracy: 0.7785
Epoch 1838/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3715 - accuracy: 0.7940
Epoch 1839/2000
18/18 [==============================] - 0s 5ms/step - loss: 1.4846 - accuracy: 0.7657
Epoch 1840/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3554 - accuracy: 0.7958
Epoch 1841/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.3841 - acc

18/18 [==============================] - 0s 4ms/step - loss: 1.7732 - accuracy: 0.8304
Epoch 1912/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.9033 - accuracy: 0.8213
Epoch 1913/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.9997 - accuracy: 0.8241
Epoch 1914/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.9297 - accuracy: 0.8250
Epoch 1915/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.0386 - accuracy: 0.8186
Epoch 1916/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.9771 - accuracy: 0.8168
Epoch 1917/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.9009 - accuracy: 0.8204
Epoch 1918/2000
18/18 [==============================] - 0s 4ms/step - loss: 2.0208 - accuracy: 0.8149
Epoch 1919/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.9173 - accuracy: 0.8277
Epoch 1920/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.9255 - acc

18/18 [==============================] - 0s 4ms/step - loss: 1.7768 - accuracy: 0.8104
Epoch 1991/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.7838 - accuracy: 0.7821
Epoch 1992/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.8118 - accuracy: 0.7995
Epoch 1993/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.8395 - accuracy: 0.7949
Epoch 1994/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.8418 - accuracy: 0.7739
Epoch 1995/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.7767 - accuracy: 0.7931
Epoch 1996/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.8311 - accuracy: 0.7840
Epoch 1997/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.7174 - accuracy: 0.8140
Epoch 1998/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.8142 - accuracy: 0.7739
Epoch 1999/2000
18/18 [==============================] - 0s 4ms/step - loss: 1.8819 - acc

In [13]:
test_loss_error_real, test_acc_error_real = model_error.evaluate(xtest, ytest)

9/9 [==============================] - 0s 3ms/step - loss: 1.1272 - accuracy: 0.8364


In [20]:
test_loss_real_error, test_acc_real_error = model.evaluate(xtest_error, ytest_error)

9/9 [==============================] - 0s 2ms/step - loss: 1.2461 - accuracy: 0.8655


In [15]:
lq.models.summary(model)

+sequential stats--------------------------------------------------------------------+
| Layer                  Input prec.  Outputs  # 1-bit  # 32-bit  Memory  1-bit MACs |
|                              (bit)               x 1       x 1    (kB)             |
+------------------------------------------------------------------------------------+
| quant_dense                      1  (-1, 4)       16         0    0.00          16 |
| batch_normalization              -  (-1, 4)        0         8    0.03           0 |
| quant_dense_1                    1  (-1, 8)       32         0    0.00          32 |
| batch_normalization_1            -  (-1, 8)        0        16    0.06           0 |
| quant_dense_2                    1  (-1, 4)       32         0    0.00          32 |
| batch_normalization_2            -  (-1, 4)        0         8    0.03           0 |
| quant_dense_3                    1  (-1, 1)        4         0    0.00           4 |
| batch_normalization_3            -  (-1, 

In [16]:
#The output is still accuarate with any added variation in data.
print(f"The accuracy with no added random error is: {test_acc * 100:.2f} %")

The accuracy with no added random error is: 88.00 %


In [17]:
print(f"The accuracy with added random error is:  {test_acc_error * 100:.2f} %")

The accuracy with added random error is:  81.45 %


In [18]:
print(f"The accuracy with added random on real set is:  {test_acc_error_real * 100:.2f} %")

The accuracy with added random on real set is:  83.64 %


In [21]:
print(f"The accuracy with added random on real set is:  {test_acc_real_error * 100:.2f} %")

The accuracy with added random on real set is:  86.55 %
